In [122]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.Strategy.MVO import *
from strategy_v2.Strategy.MVO.AlphaModel import *
from strategy_v2.Strategy.MVO.RiskModel import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
from utils.ta import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',    
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ',
    'NANC',
    #'BTC',    
    'CASH'
]
end_date = get_today(0)
start_date = pd.to_datetime(datetime(2024, 1, 3))
start_date = pd.to_datetime(datetime(2023, 2, 7))
max_leverage = 1
#vol_target = 0.25
vol_target = None
end_date

Timestamp('2025-03-28 00:00:00')

# Notes

- 2024-08-28: Tested expected return prediction with RandomForest, XGB, LGBM. None of them outperforms the SMA Model. LGBM is able to achieve a similar performance as SMA model and generally train faster.</br>

    | Measure                | ^SPX      | MVO - SMA1 | MVO - RandomForest1 | MVO - XGB1 | MVO - LGBM1 |
    |------------------------|-----------|------------|---------------------|------------|-------------|
    | Cumulative Return      | 1.173503  | 2.072994   | 1.662573            | 1.471717   | 2.023378    |
    | Annualized Return      | 0.076775  | 0.301537   | 0.221185            | 0.176047   | 0.296252    |
    | Annualized Volatility  | 0.180081  | 0.226417   | 0.240133            | 0.243953   | 0.243120    |
    | Annualized Sharpe Ratio| 0.240438  | 1.183923   | 0.781685            | 0.584416   | 1.080844    |
    | Maximum Drawdown       | -0.254251 | -0.271732  | -0.323658           | -0.380737  | -0.296535   |

    Models are shared the same hyperparameters: lookback (train days) = 10 days and gamma=10, hhi=0.2


## Leverage 
##### 1. capital (Stock MV / last leverage), 2. new leverage
2024-09-06: $169,336, 96.69%

2024-09-20: $172,168, 93.52%

2024-10-04: $176,235, 100%

2024-10-18: $177,831, 100%

2024-11-01: $175,558, 100%

2024-11-09: $181,511

2024-11-22: $187,224

2024-11-30: $188,332

2024-12-06: $194,828

2024-12-13: $211,354, CASH: $3778

2024-12-20: $198,713, CASH: $38,024 (+$20000 Capital)

2025-01-04: $192,246, CASH: $23,606

2024-01-10: $205,516, CASH: $18,562

2025-01-17: $213,405, CASH: $18,835

2025-01-24: $216,851, CASH: $11,204

2025-01-31: $228,709, CASH: $4,254

2025-02-07: $232,687, CASH: $9,159

2025-02-14: $230,291, CASH: $5,880

2025-02-21: $232,495, CASH: $5,202

2025-02-28: $214,062, CASH: $32,893

2025-03-07: $183,794, CASH: $44,484

2025-03-14: $170,198, CASH: $61,726

2025-03-21: $157,754, CASH: $21,438

2025-03-28: $194,943, CASH: $26,857

# Log

2024-11-22: If we don't fix sum of weights to constant (e.g. 1), the model would statisfy the HHI constraints by reducing the overeall weights only, but not diversing the portfolios. Still, fixing to 1 is not good because sometime we should de-leverage given signals are weak. So we can consider to add a cash element in the optimization

2024-11-30: Create benchmark portfolios (e.g. equally weighted of all stocks) (DONE)

2024-12-06: Alpha models using returns relative to something (e.g. SPX)

In [132]:
portfolio = PortfolioStandard(
    capital=194943 + 21438,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 1),        
    tc_model=TransactionCostFutu(), 
    systems_style=SystemStyle.VERTICAL,
    systems=[        
        # Long terms signals => 60days return / diversified        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(60), risk_model=RollingMeanCovNeg(60), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Mid terms signals => 30days return / diversified        
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(30), risk_model=RollingMeanCovNeg(30), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Short terms signals => 5 and 10days returns / less diversified
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(10), risk_model=RollingMeanCovNeg(10), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(5), risk_model=RollingMeanCovNeg(5), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         

        # Mean Revert signals => 2days RSI / concentrated
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,10), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.2)], max_leverage=max_leverage, offset=200),
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,95), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=-1)], max_leverage=max_leverage, offset=200),

        # Mean Revert signals => Double7 / concentrated
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=Double7(5), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.2)], max_leverage=max_leverage, offset=200),
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2025-03-29 02:36:09,507 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Generating position for strategy MVO - RollingMean(60)|1 between 2023-02-06 and 2025-03-28......
100%|██████████| 560/560 [00:03<00:00, 149.54it/s]
2025-03-29 02:36:13,273 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Volatility Target = nan% | Price Volatility = 16.9% | Last Scale Factor = 1.00
2025-03-29 02:36:13,275 - TradingSubSystemSingle [MVO - RollingMean(10)|0.5] - INFO - Generating position for strategy MVO - RollingMean(10)|0.5 between 2023-02-06 and 2025-03-28......
100%|██████████| 560/560 [00:03<00:00, 160.89it/s]
2025-03-29 02:36:16,769 - TradingSubSystemSingle [MVO - RollingMean(10)|0.5] - INFO - Volatility Target = nan% | Price Volatility = 9.3% | Last Scale Factor = 1.00
2025-03-29 02:36:16,775 - TradingSubSystemSingle [MVO - Double(5)|0.2] - INFO - Generating position for strategy MVO - Double(5)|0.2 between 2023-02-06 and 2025-03-28......
100%|██████████| 560/560 [00:01<

,Rebalanced Portfolio,Optimized Portfolio,^SPX,Equal Weighted,MVO - RollingMean(60)|1,MVO - RollingMean(10)|0.5,MVO - Double(5)|0.2
Measure,,,,,,,
Cumulative Return,2.742783,2.491132,1.343636,1.933128,2.504729,1.624437,1.275957
Annualized Return,0.501966,0.457815,0.147194,0.327499,0.460914,0.236069,0.117682
Annualized Volatility,0.237872,0.242080,0.130621,0.189752,0.244104,0.128949,0.081480
Annualized Sharpe Ratio,1.991232,1.774241,0.910162,1.576751,1.772226,1.611199,1.096887
Maximum Drawdown,-0.157577,-0.161082,-0.102766,-0.134150,-0.178395,-0.111620,-0.052489


In [125]:
portfolio.portfolio_breakdown()
portfolio.instrument_breakdown()

In [126]:
portfolio.get_position_for_trade().tail(20)

2025-03-29 02:17:45,917 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $216,381
2025-03-29 02:17:45,917 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $216,381


,AAPL,BRK-B,DXJ,META,NANC,NVDA,QQQ,SPY,TSLA,CASH
Date,,,,,,,,,,
2025-03-03,86.0,118.0,263.0,65.0,43.0,0.0,13.0,9.0,90.0,27008.0
2025-03-04,86.0,118.0,263.0,65.0,43.0,0.0,13.0,9.0,90.0,27008.0
2025-03-05,86.0,118.0,263.0,65.0,43.0,0.0,13.0,9.0,90.0,27008.0
2025-03-06,86.0,118.0,263.0,65.0,43.0,0.0,13.0,9.0,90.0,27008.0
2025-03-07,50.0,138.0,303.0,83.0,0.0,0.0,3.0,8.0,0.0,44422.0
2025-03-10,50.0,138.0,303.0,83.0,0.0,0.0,3.0,8.0,0.0,44422.0
2025-03-11,50.0,138.0,303.0,83.0,0.0,0.0,3.0,8.0,0.0,44422.0
2025-03-12,50.0,138.0,303.0,83.0,0.0,0.0,3.0,8.0,0.0,44422.0
2025-03-13,50.0,138.0,303.0,83.0,0.0,0.0,3.0,8.0,0.0,44422.0


# Execute the Portfolio

In [87]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2025-03-29 01:42:30,000 - ExecutorFutu - INFO - market: US
2025-03-29 01:42:30,009 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2025-03-29 01:42:30,024 | 16174 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=11, host=127.0.0.1, port=11111, user_id=18214795
2025-03-29 01:42:30,163 | 16174 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=11


2025-03-29 01:42:33,164 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $216,381
2025-03-29 01:42:33,165 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $216,381
2025-03-29 01:42:33,170 - ExecutorFutu - INFO - Execute Standard Portfolio ({self.name}) position based on 2025-03-28


2025-03-29 01:42:33,174 | 16174 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=12, host=127.0.0.1, port=11111, user_id=18214795


2025-03-29 01:42:33,190 - Futu - INFO - 8 Positions: US.EWY, US.SPY, US.QQQ, US.NVDA, US.NANC, US.META, US.DXJ, US.BRK.B


...

2025-03-29 01:42:54,148 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 555.64, 'qty': 1.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2025-03-29 01:42:54,150 | 16174 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=19
2025-03-29 01:42:57,165 | 16174 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=20, host=127.0.0.1, port=11111, user_id=18214795


2025-03-29 01:42:57,458 - Futu - INFO - Placed Order: {'code': 'US.TSLA', 'price': 262.98, 'qty': 34.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2025-03-29 01:42:57,459 | 16174 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=20


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.AAPL,苹果,BUY,NORMAL,SUBMITTING,FH1A72530C2A9B6000,64.0,218.27,2025-03-28 13:42:34,2025-03-28 13:42:34,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28
0,US.BRK.B,伯克希尔-B,SELL,NORMAL,SUBMITTING,FH1A72530F5ADB6000,3.0,525.50,2025-03-28 13:42:37,2025-03-28 13:42:37,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28
0,US.DXJ,日本对冲股票ETF-WisdomTree,BUY,NORMAL,SUBMITTING,FH1A725312A17F2000,63.0,110.74,2025-03-28 13:42:41,2025-03-28 13:42:41,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28
0,US.META,Meta Platforms,SELL,NORMAL,SUBMITTING,FH1A725315D1AE8000,41.0,581.20,2025-03-28 13:42:44,2025-03-28 13:42:44,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28
0,US.NANC,UNUSUAL WHALES SUBVERSIVE DEMOCRATIC TRADING ETF,BUY,NORMAL,SUBMITTING,FH1A725319062E8000,11.0,35.87,2025-03-28 13:42:47,2025-03-28 13:42:47,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28
0,US.NVDA,英伟达,SELL,NORMAL,SUBMITTING,FH1A72531C3C7F2000,136.0,109.80,2025-03-28 13:42:50,2025-03-28 13:42:50,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28
0,US.SPY,标普500ETF-SPDR,BUY,NORMAL,SUBMITTING,FH1A72531F76AE8000,1.0,555.64,2025-03-28 13:42:54,2025-03-28 13:42:54,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28
0,US.TSLA,特斯拉,BUY,NORMAL,SUBMITTING,FH1A725322AAF81000,34.0,262.98,2025-03-28 13:42:57,2025-03-28 13:42:57,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-03-28


# Calibration Portfolios